In [1]:
from src.utils import visualisations
from src.utils.utils import DataTransformer
from torch_geometric.data import Data
from src.datasets.zinc_smiles_generation import ZincSmiles
from src.datasets.zinc_pairs_v2 import ZincPairsV2
from src.encoding.decoder import is_induced_subgraph_by_features


train_ds = ZincPairsV2(split="train", base_dataset=ZincSmiles(split="train"), dev=True)
test_ds = ZincPairsV2(split="test", base_dataset=ZincSmiles(split="train"), dev=True)
valid_ds = ZincPairsV2(split="valid", base_dataset=ZincSmiles(split="train"), dev=True)
counter = 1
for item in [*train_ds, *test_ds, *valid_ds]:
    g1 = Data(x=item.x1, edge_index=item.edge_index1)
    g2 = Data(x=item.x2, edge_index=item.edge_index2)
    y = bool(item.y.item())

    g1_x = DataTransformer.pyg_to_nx(g1)
    g2_x = DataTransformer.pyg_to_nx(g2)
    is_induced = is_induced_subgraph_by_features(g1_x, g2_x, node_keys=["feat"])

    if is_induced != y:
        print(f"[{counter}] WRONG")
        visualisations.draw_nx_with_atom_colorings(g1_x, label=f"[{counter}]Actual: {is_induced}, Y: {y}")
        visualisations.draw_nx_with_atom_colorings(g2_x, label=f"[{counter}]Actual: {is_induced}, Y: {y}")
    else:
        print(f"[{counter}] OK")
    counter += 1

[1] OK
[2] OK
[3] OK
[4] OK
[5] OK
[6] OK
[7] OK
[8] OK
[9] OK
[10] OK
[11] OK
[12] OK
[13] OK
[14] OK
[15] OK
[16] OK
[17] OK
[18] OK
[19] OK
[20] OK
[21] OK
[22] OK
[23] OK
[24] OK
[25] OK
[26] OK
[27] OK
[28] OK
[29] OK
[30] OK
[31] OK
[32] OK
[33] OK
[34] OK
[35] OK
[36] OK
[37] OK
[38] OK
[39] OK
[40] OK
[41] OK
[42] OK
[43] OK
[44] OK
[45] OK
[46] OK
[47] OK
[48] OK
[49] OK
[50] OK
[51] OK
[52] OK
[53] OK
[54] OK
[55] OK
[56] OK
[57] OK
[58] OK
[59] OK
[60] OK
[61] OK
[62] OK
[63] OK
[64] OK
[65] OK
[66] OK
[67] OK
[68] OK
[69] OK
[70] OK
[71] OK
[72] OK
[73] OK
[74] OK
[75] OK
[76] OK
[77] OK
[78] OK
[79] OK
[80] OK
[81] OK
[82] OK
[83] OK
[84] OK
[85] OK
[86] OK
[87] OK
[88] OK
[89] OK
[90] OK
[91] OK
[92] OK
[93] OK
[94] OK
[95] OK
[96] OK
[97] OK
[98] OK
[99] OK
[100] OK
[101] OK
[102] OK
[103] OK
[104] OK
[105] OK
[106] OK
[107] OK
[108] OK
[109] OK
[110] OK
[111] OK
[112] OK
[113] OK
[114] OK
[115] OK
[116] OK
[117] OK
[118] OK
[119] OK
[120] OK
[121] OK
[122] OK
[123] OK
[